In [1]:
import pandas as pd
import os
from termcolor import colored
run_setting_f=open("run_setting_nextseq.csv")
run_config_dict={}
sample_dict={}
df=pd.DataFrame(columns=['sample_id','concentration','pct','pct/con'])

for line in run_setting_f:
    if line.startswith("sample ID"):
        break
    items=line.split(",")
    run_config_dict[items[0]]=items[1]
    
for line in run_setting_f:
    items=line.strip().split(",")
    print (items)
    if items[1]=='':
        df=df.append({'sample_id':items[0],'concentration':items[1],'pct':float(items[2])},ignore_index=True)
    else:
        df=df.append({'sample_id':items[0],'concentration':float(items[1]),'pct':float(items[2]),
                                                             'pct/con':float(items[2])/float(items[1])},ignore_index=True)
    
file_name='%s_Nextseq_%s_%s+%s_%s.xlsx' %(run_config_dict['date'],run_config_dict['HO/MO'],run_config_dict['R1'],
                                   run_config_dict['R2'],run_config_dict['project_id'])


#final loading volume
size=int(run_config_dict['size of library'])
target_cluster_density=int(run_config_dict['target_cluster_density'])

if size<=250:
    loading_pM=target_cluster_density/185
    phix_cluster_efficiency=1.45
elif size<350:
    loading_pM=target_cluster_density/175
    phix_cluster_efficiency=1.8
elif size<450:
    loading_pM=target_cluster_density/165
    phix_cluster_efficiency=2.6
elif size>=450:
    loading_pM=target_cluster_density/155
    phix_cluster_efficiency=3
    
    

final_loading_16pM_sample=1300*float(run_config_dict['% of library'])*loading_pM/16
final_loading_20pM_phix=target_cluster_density*float(run_config_dict['% of phix'])/phix_cluster_efficiency
HT1=1300-final_loading_16pM_sample-final_loading_20pM_phix



    
    

#to check if the total pct over 100
if df['pct'].sum() >102 or df['pct'].sum()<95:
    print (colored('pencentage is wrong','red'))
    

    


['1735FL-1808-01', '', '100']


In [2]:
#if there is only one sample and only do not have qPCR result yet
from openpyxl import load_workbook

if len(df)==1 and df['concentration'][0]=='':
    wb_loading=load_workbook("./Nextseq_loading_template.xlsx")
    ws_loading=wb_loading['loading']
    title="Nextseq_%s_%s+%s+%s+%s" %(run_config_dict['HO/MO'],run_config_dict['R1'],run_config_dict['I1'],
                              run_config_dict['I2'],run_config_dict['R2'])
    ws_loading.cell(row=1,column=1).value=title
    ws_loading.cell(row=4,column=2).value=df['sample_id'][0]
    ws_loading.cell(row=4,column=4).value='=IF(C4>16,2,1.6*10/C4)'
    ws_loading.cell(row=4,column=5).value='=D4*C4/1.6-D4'
    ws_loading.cell(row=4,column=6).value='10ul 1.6nM sample+10ul 0.1 NaOH+980ul HT1'
    ws_loading.cell(row=4,column=7).value=final_loading_16pM_sample
    ws_loading.cell(row=4,column=8).value=final_loading_20pM_phix
    ws_loading.cell(row=4,column=9).value=HT1
    
    row_now=5
    
    #customer primer 
    if run_config_dict['customer/spike-in']== '':
        pass
    else:
        read1_primer=run_config_dict['read1 primer']
        read2_primer=run_config_dict['read2 primer']
        index1_primer=run_config_dict['index1 primer']
        index2_primer=run_config_dict['index2 primer']

        #spike in primer
        if run_config_dict['customer/spike-in']=='spike-in':
            if read1_primer !='':
                if run_config_dict['HO/MO']=='MO':
                    read1_primer_add='4ul 100uM Read1 primer into #20, DO NOT SELECT ANYTHING ON MACHINE!'
                elif run_config_dict['HO/MO']=='HO':
                    read1_primer_add='5.2ul 100uM Read1 primer into #20, DO NOT SELECT ANYTHING ON MACHINE!'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read1_primer
                ws_loading.cell(row=row_now,column=6).value=read1_primer_add    

            if read2_primer !='':
                if run_config_dict['HO/MO']=='MO':
                    read2_primer_add='4.5ul 100uM Read2 primer into #21,DO NOT SELECT ANYTHING ON MACHINE!'
                elif run_config_dict['HO/MO']=='HO':
                    read2_primer_add='6ul 100uM Read2 primer into #21,DO NOT SELECT ANYTHING ON MACHINE!'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read2_primer
                ws_loading.cell(row=row_now,column=6).value=read2_primer_add            

            if index1_primer !='' or index2_primer !='':
                if run_config_dict['HO/MO']=='MO':
                    if index1_primer !='' and index2_primer !='':
                        index_primer=index1_primer+' and '+index2_primer
                        index_primer_add='3ul 100uM Index1 primer and 3ul 100uM Index2 primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'
                    else:
                        index_primer=index1_primer+index2_primer
                        index_primer_add='6ul 100uM Index primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'

                elif  run_config_dict['HO/MO']=='HO':
                    if index1_primer !='' and index2_primer !='':
                        index_primer=index1_primer+' and '+index2_primer
                        index_primer_add='4.3ul 100uM Index1 primer and 4.3ul 100uM Index2 primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'
                    else:
                        index_primer=index1_primer+index2_primer
                        index_primer_add='8.5ul 100uM Index primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'                   
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=index_primer
                ws_loading.cell(row=row_now,column=6).value=index_primer_add
        #customer primer        
        elif run_config_dict['customer/spike-in']=='customer':
            if read1_primer !='':
                read1_primer_add='6ul 100uM read 1 primer+1994ul HT1-->#7, SELECT READ 1 ON MACHINE!'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read1_primer
                ws_loading.cell(row=row_now,column=6).value=read1_primer_add    

            if read2_primer !='':
                read2_primer_add='6ul 100uM read 2 primer+1994ul HT1-->#8, SELECT READ 2 ON MACHINE!'
                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=read2_primer
                ws_loading.cell(row=row_now,column=6).value=read2_primer_add            

            if index1_primer !='' or index2_primer !='':
                if index1_primer !='' and index2_primer !='':
                    index_primer=index1_primer+' and '+index2_primer
                    index_primer_add='3ul 100uM index primer 1+3ul 100uM index primer 2+1994ul HT1-->#9, SELECT INDEX ON MACHINE!'
                else:
                    index_primer=index1_primer+index2_primer
                    index_primer_add='6ul 100uM index primer+1994ul HT1-->#9, SELECT INDEX ON MACHINE!'

                row_now+=1
                ws_loading.cell(row=row_now,column=5).value=index_primer
                ws_loading.cell(row=row_now,column=6).value=index_primer_add
        elif run_config_dict['customer/spike-in'] is not None:
            print (colored("customer primer is not writing correctly","red"))



    row_now+=2
    ws_loading.cell(row=row_now,column=5).value='Box'
    ws_loading.cell(row=row_now,column=6).value=run_config_dict['sample pos']
    wb_loading.save(file_name)
    os.startfile(file_name)

In [3]:
#divied the sample into different pools
dict_of_df={}

low_pct_con=min(df['pct/con'])
for i in range(1,10):
    if len(df)==0:
        break
    #based on pct/con value, divided into different pool
    dict_of_df['df_pool%s' %i]=[df[df['pct/con']<=low_pct_con*10].reset_index(drop=True),0]
    
    #the rest sample will be in another pool
    df=df[df['pct/con']>low_pct_con*10]
    try:
        low_pct_con=min(df['pct/con'])
    except:
        break

for df_name,df_itself_and_water in dict_of_df.items():
    if len(df_itself_and_water[0])>1:
        low_pct_con=10000000000
        total_mol=0
        for i,j in df_itself_and_water[0].sort_values(by='pct/con',ascending=True).iterrows():
            print (i)
            low_pct_con=min(low_pct_con,j['pct/con'])
            df_itself_and_water[0].at[i,'add vol']=round(j['pct/con']/low_pct_con,2)
            total_mol+=j['pct/con']/low_pct_con*j['concentration']
            print ('add value = %s' %(j['pct/con']/low_pct_con) )

        #water volumn addded
        water_vol=total_mol/1.6-df_itself_and_water[0]['add vol'].sum()
        dict_of_df[df_name][1]=water_vol
        print (water_vol)

    elif len(df_itself_and_water[0])==1:
        if df_itself_and_water[0]['concentration'][0] > 16:
            df_itself_and_water[0].at[0,'add vol']=2
            water_vol=1.6*df_itself_and_water[0]['concentration'][0]/1.6-2
        else:
            df_itself_and_water[0].at[0,'add vol']=16/df_itself_and_water[0]['concentration'][0]
            water_vol=10-16/df_itself_and_water[0]['concentration'][0]
        dict_of_df[df_name][1]=water_vol

In [4]:
#generate excel file  
from openpyxl.styles.borders import Border, Side
from openpyxl.styles import Alignment
from openpyxl import Workbook
from openpyxl import load_workbook

from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter

wb_loading=load_workbook("./Nextseq_loading_template.xlsx")
ws_loading=wb_loading['loading']

thin = Side(border_style="thin", color="000000")

title="Nextseq_%s_%s+%s+%s+%s" %(run_config_dict['HO/MO'],run_config_dict['R1'],run_config_dict['I1'],
                              run_config_dict['I2'],run_config_dict['R2'])
ws_loading.cell(row=1,column=1).value=title

row_start=4
row_now=3
for df_name,df_itself_and_water in dict_of_df.items():
    df_itself=df_itself_and_water[0]
    pool_id=run_config_dict['project_id']+'-loading-'+df_name.split('_')[1]
    ws_loading.cell(row=row_start,column=1).value=pool_id
    
    for index,sample in df_itself.iterrows():
        row_now+=1   
        ws_loading.cell(row=row_now,column=2).value=sample['sample_id']
        ws_loading.cell(row=row_now,column=3).value=sample['concentration']
        ws_loading.cell(row=row_now,column=4).value=sample['add vol']
    
    ws_loading.cell(row=row_start,column=5).value=df_itself_and_water[1]
    ws_loading.cell(row=row_start,column=6).value='10ul 1.6nM sample+10ul 0.1 NaOH+980ul HT1'
    ws_loading.cell(row=row_start,column=7).value=final_loading_16pM_sample*df_itself['pct'].sum()/100
    #merge the cell for different pool
    for col,col_num in zip(['A','E','F','G'],[1,5,6,7]):
        #print (col,col_num)
        ws_loading.merge_cells("%s%s:%s%s" %(col,row_start,col,row_now))
        ws_loading.cell(row=row_start,column=col_num).alignment=Alignment(horizontal='center',vertical='center')
    
    row_now+=1
    row_start=row_now+1

#merge the cell for final loading part

ws_loading.cell(row=4,column=8).value=final_loading_20pM_phix
ws_loading.cell(row=4,column=9).value=HT1

for col,col_num in zip(['H','I'],range(8,10)):
    print (col,col_num)
    ws_loading.merge_cells("%s4:%s%s" %(col,col,row_now-1))
    ws_loading.cell(row=4,column=col_num).alignment=Alignment(horizontal='center',vertical='center')

for row in ws_loading['A%s:I%s' %(1,row_now-1)]:
    for cell in row:
        cell.border=Border(top=thin,left=thin,right=thin,bottom=thin)
        

row_now+=1
#customer primer 
if run_config_dict['customer/spike-in']== '':
    pass
else:
    read1_primer=run_config_dict['read1 primer']
    read2_primer=run_config_dict['read2 primer']
    index1_primer=run_config_dict['index1 primer']
    index2_primer=run_config_dict['index2 primer']
    
    #spike in primer
    if run_config_dict['customer/spike-in']=='spike-in':
        if read1_primer !='':
            if run_config_dict['HO/MO']=='MO':
                read1_primer_add='4ul 100uM Read1 primer into #20, DO NOT SELECT ANYTHING ON MACHINE!'
            elif run_config_dict['HO/MO']=='HO':
                read1_primer_add='5.2ul 100uM Read1 primer into #20, DO NOT SELECT ANYTHING ON MACHINE!'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read1_primer
            ws_loading.cell(row=row_now,column=6).value=read1_primer_add    
            
        if read2_primer !='':
            if run_config_dict['HO/MO']=='MO':
                read2_primer_add='4.5ul 100uM Read2 primer into #21,DO NOT SELECT ANYTHING ON MACHINE!'
            elif run_config_dict['HO/MO']=='HO':
                read2_primer_add='6ul 100uM Read2 primer into #21,DO NOT SELECT ANYTHING ON MACHINE!'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read2_primer
            ws_loading.cell(row=row_now,column=6).value=read2_primer_add            
            
        if index1_primer !='' or index2_primer !='':
            if run_config_dict['HO/MO']=='MO':
                if index1_primer !='' and index2_primer !='':
                    index_primer=index1_primer+' and '+index2_primer
                    index_primer_add='3ul 100uM Index1 primer and 3ul 100uM Index2 primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'
                else:
                    index_primer=index1_primer+index2_primer
                    index_primer_add='6ul 100uM Index primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'
                    
            elif  run_config_dict['HO/MO']=='HO':
                if index1_primer !='' and index2_primer !='':
                    index_primer=index1_primer+' and '+index2_primer
                    index_primer_add='4.3ul 100uM Index1 primer and 4.3ul 100uM Index2 primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'
                else:
                    index_primer=index1_primer+index2_primer
                    index_primer_add='8.5ul 100uM Index primer into #22,DO NOT SELECT ANYTHING ON MACHINE!'                   
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=index_primer
            ws_loading.cell(row=row_now,column=6).value=index_primer_add
    #customer primer        
    elif run_config_dict['customer/spike-in']=='customer':
        if read1_primer !='':
            read1_primer_add='6ul 100uM read 1 primer+1994ul HT1-->#7, SELECT READ 1 ON MACHINE!'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read1_primer
            ws_loading.cell(row=row_now,column=6).value=read1_primer_add    
            
        if read2_primer !='':
            read2_primer_add='6ul 100uM read 2 primer+1994ul HT1-->#8, SELECT READ 2 ON MACHINE!'
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=read2_primer
            ws_loading.cell(row=row_now,column=6).value=read2_primer_add            
            
        if index1_primer !='' or index2_primer !='':
            if index1_primer !='' and index2_primer !='':
                index_primer=index1_primer+' and '+index2_primer
                index_primer_add='3ul 100uM index primer 1+3ul 100uM index primer 2+1994ul HT1-->#9, SELECT INDEX ON MACHINE!'
            else:
                index_primer=index1_primer+index2_primer
                index_primer_add='6ul 100uM index primer+1994ul HT1-->#9, SELECT INDEX ON MACHINE!'
 
            row_now+=1
            ws_loading.cell(row=row_now,column=5).value=index_primer
            ws_loading.cell(row=row_now,column=6).value=index_primer_add
    elif run_config_dict['customer/spike-in'] is not None:
        print (colored("customer primer is not writing correctly","red"))
        
       
    
row_now+=2
ws_loading.cell(row=row_now,column=5).value='Box'
ws_loading.cell(row=row_now,column=6).value=run_config_dict['sample pos']



wb_loading.save(file_name)
os.startfile(file_name)


ValueError: 3 must be greater than 4

In [5]:
sample_sheet_template_f=open('./SampleSheet_nextseq_template.csv')
output_sample_sheet=open('%s_%s_SampleSheet.csv' %(run_config_dict['date'],run_config_dict['project_id']),'w+')

for index,line in enumerate(sample_sheet_template_f):
    if index==2:
        output_sample_sheet.write("Date,%s\n" %run_config_dict['date'])
    elif index==6:
        output_sample_sheet.write("Description,%s\n" %run_config_dict['project_id'])
    elif index==10:
        output_sample_sheet.write('%s\n' %(run_config_dict['R1']))
    elif index==11:
        if run_config_dict['R2'] !='0':
            output_sample_sheet.write("%s\n" %run_config_dict['R2'] )
    else:
        output_sample_sheet.write(line)
sample_sheet_template_f.close()
output_sample_sheet.close()

os.startfile('%s_%s_SampleSheet.csv' %(run_config_dict['date'],run_config_dict['project_id']))